<a href="https://colab.research.google.com/github/JiaminJIAN/20MA573/blob/master/src/Euler_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy.optimize as so
import scipy.stats as ss
import matplotlib.pyplot as plt
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

In [44]:
#Read four-column data
#columns are otype, maturity, strike, option_price
np_option_data = np.loadtxt('20optiondata2.dat', comments='#', delimiter=',')

print('>>>>>>otype, maturity, strike, option_price')
print(np_option_data)

>>>>>>otype, maturity, strike, option_price
[[  1.           0.16666667  97.           5.32705461]
 [  1.           0.16666667  99.           3.86224255]
 [  1.           0.16666667 101.           2.7204371 ]
 [  1.           0.16666667 103.           2.1202793 ]
 [  1.           0.41666667  97.           7.23756307]
 [  1.           0.41666667  99.           5.95053461]
 [  1.           0.41666667 101.           5.2640122 ]
 [  1.           0.41666667 103.           4.97493422]]


In [0]:
'''=========
option class init
=========='''
class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price #this will be used for calibration
      
        
    def payoff(self, s): #s: excercise price
      otype = self.otype
      k = self.strike
      maturity = self.maturity
      return max([0, (s - k)*otype])

In [0]:
num_row = np_option_data.shape[0]
option_list = []

for i in range(num_row):
  option1 = VanillaOption(
      otype = np_option_data[i,0],
      strike = np_option_data[i,2],
      maturity = np_option_data[i,1],
      market_price = np_option_data[i,3]
  )   
  option_list.append(option1)

In [47]:
print(option_list)

[<__main__.VanillaOption object at 0x7fe7269d3438>, <__main__.VanillaOption object at 0x7fe7269d36a0>, <__main__.VanillaOption object at 0x7fe7269d3390>, <__main__.VanillaOption object at 0x7fe7269d3898>, <__main__.VanillaOption object at 0x7fe7269d3978>, <__main__.VanillaOption object at 0x7fe7269d3e80>, <__main__.VanillaOption object at 0x7fe726a37080>, <__main__.VanillaOption object at 0x7fe726a37a58>]


In [0]:
class euler:
    def __init__(self, mu, sigma, gamma, InitState):
        self.mu = mu
        self.sigma = sigma
        self.gamma = gamma
        self.InitState = InitState

    def Mu(self, mu):
        mu = self.mu
        Mu = lambda x,t: mu*x

        return Mu

    def Sigma(self, sigma, gamma):
        sigma = self.sigma
        gamma = self.gamma
        Sigma = lambda x,t: sigma*x**gamma

        return Sigma

        
    def Euler_method(self, T, N):
        x0 = self.InitState     
        t = np.linspace(0, T, N+1)
        
        Wh = np.zeros(N+1)   ##initial BM
        Sh =s0 + np.zeros(N+1)   ##initial Xh
        
        for i in range(N):  ##the Euler method
            DeltaT = t[i+1] - t[i]
            DeltaW = np.sqrt(t[i+1] - t[i]) * np.random.normal()
            Wh[i+1] = Wh[i] + DeltaW
            Sh[i+1] =Sh[i] + Mu(Sh[i], t[i]) * DeltaT + \
            Sigma(Sh[i], t[i])* DeltaW
            
        return t, Sh, Wh

    def call_payoff(self, mu, S_T):
        mu = self.mu
        k = vanilla_option.strike
        maturity = vanilla_option.maturity
        return np.exp(-r*T)*max(0,S_T - k)


def option_price(self, vanilla_option):
        x0 = self.InitState 
    
        otype = vanilla_option.otype
        k = vanilla_option.strike
        maturity = vanilla_option.maturity
    
        for i in range(IterN):
          [t, Sh, Wh] = Euler_method(maturity, 1000);
          call_price_list[i] = call_payoff(mu, Sh[-1])
    
        return call_price_list.mean()

euler.option_price = option_price

In [0]:
euler1 = euler(
    mu = 0.03,
    sigma = 0.2,
    gamma = 1,
    InitState = 100
)

In [0]:
'''==========
define a method for error function
============'''

def error_function(sigma, gamma, euler, option_list):
  euler.sigma = sigma
  euler.gamma = gamma
  error = 0
  for i in np.arange(len(option_list)):
    error = error + ((option_list[i]).market_price - euler.option_price(option_list[i]))**2    
  return error


'''==========
define a method to seek for a calibrated volatility
============'''

def cev_calibration(euler, option_list):
  sigma = 0.2    #initial guess
  gamma = 1      #initial guess

  return so.fmin(error_function, sigma , args = (euler, option_list), disp = 0)[0]

In [51]:
# Take options of 5-months maturity

filter1 = list(filter(lambda x: x.maturity == 5/12, option_list))
calibrated_parameters = cev_calibration(euler1, filter1)
print('calibrated_parameters is' ,calibrated_parameters)

TypeError: ignored